# Цель: сгенерировать текст стилем автора, используя предобученную LLM

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch as th
import re

from google.colab import drive
from google.colab import files

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

from warnings import filterwarnings
filterwarnings('ignore')

sns.set_style('darkgrid')

### Для чтения текста

In [2]:
!pip install Spire.Doc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 MB 42.6 MB/s eta 0:00:00:00:0100:01


### Очищаем .p65

In [8]:
!find . -name "*.p65" -delete

### Считываем все файлы

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


# Чтение текстов с файлов

In [32]:
import pathlib
books = pathlib.Path("/kaggle/input/bolshakov-data/new_data")
files_to_process = []
for book in books.iterdir():
    files_to_process.append(str(book))

In [40]:
files_to_process[0]

'/kaggle/input/bolshakov-data/new_data/Pril.rtf'

In [27]:
from spire.doc import *
from spire.doc.common import *

# Просмотр и обработка текстов

In [139]:
doc = Document()
doc.LoadFromFile(files_to_process[0])
text = doc.GetText()
text = text.replace('Evaluation Warning: The document was created with Spire.Doc for Python.', '')
text = text.replace('\t', '')
clean_text = re.sub(
    r'\d+\r\n|\r\n|\x0c|УДК.*|ББК.*|Издание.*|– М.*|ISBN.*|©.*',
    '\n', 
    text
)

In [88]:
clean_text[:1000]

'\nПРИЛОЖЕНИЕ\n\n1. Обратная связь как основной принцип саморегуляции систем\nи устойчивости управления\n\nВ функционировании социально-политических систем первостепенную роль играют обратные связи. Более того, любые системы с самоорганизацией способны успешно функционировать лишь при наличии соответствующего набора обратных связей. Например, одни из наиболее сложных из таких систем — социальные, биологические, экологические — основаны именно на этом принципе функционирования, как и современные автоматизированные технические устройства. Социально-политические системы, безусловно, должны быть отнесены к самоорганизующимся, так как любые имеющиеся в них органы управления являются лишь одним из элементов этих систем и в оптимальном варианте тоже подлежат организации системы “изнутри”. Следовательно, успешность функционирования общественных систем должна существенно зависеть от того, как организована в них сеть обратных связей. Можно ли говорить об оптимальной организации этих связей, суще

In [140]:
paragraphs = [p.strip() for p in clean_text.split('\n') if p.strip()]

In [147]:
paragraphs[5]

'Стимулирующие обратные связи делят на положительные и отрицательные в зависимости от того, усиливают или ослабляют они управляемый  процесс. Положительная обратная связь приводит ко все большему отклонению системы от равновесия. Отрицательная — напротив, ослабляет это. Введение последней повышает устойчивость системы, устраняет отклонения от намеченного курса, принятого в системе в качестве показателя равновесия. Образный пример работы отрицательной обратной связи в кибернетическом ее понимании — боль, которую организм испытывает, например, при прикосновении к слишком горячему предмету. Она вынуждает организм резким движением корректировать свои действия, оберегая себя от более сильного ожога и сохраняясь тем самым в равновесии. Положительная обратная связь, напротив, ускоряет процессы перехода системы из одного состояния в другое.'

# Генерирование тестовой выборки с помощью перефразеров-переводчиков



## Функция переплексии - метрика, которая вычисляет схожесть текстов по смыслу, применяя модель Сбера

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
mname = 'sberbank-ai/rugpt3small_based_on_gpt2'
gpt_tokenizer = AutoTokenizer.from_pretrained(mname)
gpt_model = AutoModelForCausalLM.from_pretrainde(mname)
gpt_model.cuda();

## Валидация работы предобученных переводчиков на одном файле 

## Полная генерация всей тестовой выборки 